In [1]:
import os
from loguru import logger
from apis.pc_apis import XHS_Apis
from xhs_utils.common_utils import init
from xhs_utils.data_util import handle_note_info, download_note, save_to_xlsx, timestamp_to_str
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time
import pandas as pd
from itertools import cycle
import logging


# logging as before…
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [2]:
class Data_Spider():
    def __init__(self):
        self.xhs_apis = XHS_Apis()

    def spider_note(self, note_url: str, cookies_str: str, proxies=None):
        """
        爬取一个笔记的信息
        :param note_url:
        :param cookies_str:
        :return:
        """
        note_info = None
        try:
            success, msg, note_info = self.xhs_apis.get_note_info(note_url, cookies_str, proxies)
            if success:
                note_info = note_info['data']['items'][0]
                note_info['url'] = note_url
                note_info = handle_note_info(note_info)
        except Exception as e:
            success = False
            msg = e
        logger.info(f'Retrieving note info - {note_url}: {success}, msg: {msg}')
        return success, msg, note_info

    def spider_some_note(self, list_notes_url: list, cookies_str: str, proxies=None):
        """
        爬取一些笔记的信息
        :param list_notes:
        :param cookies_str:
        :param base_path:
        :return:
        """
        list_note_info = []
        for note_url in list_notes_url:
            success, msg, note_info = self.spider_note(note_url, cookies_str, proxies)
            if note_info is not None and success:
                list_note_info.append(note_info)
        # for note_info in list_note_info:
        #     if save_choice == 'all' or save_choice == 'media':
        #         download_note(note_info, base_path['media'])
        # if save_choice == 'all' or save_choice == 'excel':
        #     file_path = os.path.abspath(os.path.join(base_path['excel'], f'{excel_name}.xlsx'))
        #     save_to_xlsx(list_note_info, file_path)
        return list_note_info

    def spider_some_search_note(self, query: str, require_num: int, cookies_str: str, sort="general", proxies=None):
        """
            指定数量搜索笔记，设置排序方式和笔记类型和笔记数量
            :param query 搜索的关键词
            :param require_num 搜索的数量
            :param cookies_str 你的cookies
            :param sort 排序方式 general:综合排序, time_descending:时间排序, popularity_descending:热度排序
            返回搜索的结果
        """
        time.sleep(3)
        list_note_url = []
        try:
            success, msg, list_notes = self.xhs_apis.search_some_note(query, require_num, cookies_str, sort, proxies)
            if success:
                list_notes = list(filter(lambda x: x['model_type'] == "note", list_notes))
                # logger.info(f'搜索关键词 {query} 笔记数量: {len(list_notes)}')
                logger.info(f'Searching keyword "{query}", note count: {len(list_notes)}')
                for note in list_notes:
                    note_url = f"https://www.xiaohongshu.com/explore/{note['id']}?xsec_token={note['xsec_token']}"
                    list_note_url.append(note_url)
            list_note_info = self.spider_some_note(list_note_url, cookies_str, proxies)
        except Exception as e:
            success = False
            msg = e
        # logger.info(f'搜索关键词 {query} 笔记: {success}, msg: {msg}')
        logger.info(f'Searching keyword "{query}", note: {success}, msg: {msg}')
        return success, msg, list_note_info

    def spider_search_note_all_comment(self, note_info: dict, cookies_str: str, proxies: dict = None):
        time.sleep(3)
        list_all_comment = []
        try:
            note_url = note_info["note_url"]
            success, msg, list_all_comment = self.xhs_apis.get_note_all_comment(note_url, cookies_str, proxies)
            if not success:
                raise Exception(msg)
        except Exception as e:
            success = False
            msg = e
        logger.info(f'Retrieving all comments - {note_url}: {success}, msg: {msg}')
        return success, msg, list_all_comment


In [3]:
# cookies_str, base_path = init()
# note_url = "https://www.xiaohongshu.com/explore/67b55e92000000002a00d220?xsec_token=ABB8kF8kCKc_JZwiPUZ-ii4DW8mX6djDKiG4AT2WuDyJA="
# # Data_Spider().spider_search_note_all_comment(note_url, cookie_str)
# Data_Spider().spider_note(note_url, cookies_str)
# XHS_Apis().get_note_all_comment(note_url, cookies_str)

In [4]:
# Flatten helper
def handle_list_comment(note_comment):
    list_dict_comment = []
    list_dict_subcomment = []
    for comment in note_comment:
        list_dict_comment.append(comment)
        if "sub_comments" in comment:
            for sub in comment["sub_comments"]:
                sub["comment_id"] = comment["id"]
                list_dict_subcomment.append(sub)
    return list_dict_comment, list_dict_subcomment

# Retry boilerplate
MAX_RETRIES = 3
BACKOFF_BASE = 2

def with_retry(fn):
    def wrapped(*args, **kwargs):
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                return fn(*args, **kwargs)
            except Exception as e:
                if attempt < MAX_RETRIES:
                    delay = BACKOFF_BASE ** attempt + random.random()
                    logger.warning(f"[{fn.__name__}] attempt {attempt} failed: {e!r}. retrying in {delay:.1f}s")
                    time.sleep(delay)
                else:
                    logger.error(f"[{fn.__name__}] all {MAX_RETRIES} attempts failed: {e!r}")
                    raise
    return wrapped

@with_retry
def delayed_spider_search_note(keyword, cookies_str, xhs_spider, num_search):
    # fixed 7s pre-delay
    time.sleep(7)
    success, msg, notes = xhs_spider.spider_some_search_note(keyword, num_search, cookies_str)
    if not success:
        raise RuntimeError(f"API returned success=False, msg={msg!r}")
    # fixed 7s post-delay
    time.sleep(7)
    return notes

@with_retry
def delayed_spider_search_comments(note_info, cookies_str, xhs_spider):
    # fixed 10s pre-delay
    time.sleep(10)
    success, msg, comments = xhs_spider.spider_search_note_all_comment(note_info, cookies_str)
    if not success:
        raise RuntimeError(f"API returned success=False, msg={msg!r}")
    # fixed 10s post-delay
    time.sleep(10)
    return comments

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Initialize
    xhs_spider = Data_Spider()
    cookies_str, base_path = init()

    # cookie_cycle_search = cycle(list_cookies)
    # cookie_cycle_comments = cycle(list_cookies)

    list_keywords = [
        "ifastglobalbank", "ifastgb", "ifastdebitcard",
        "ifast银行", "ifast借记卡",
    ]
    num_search = 50

    list_search_note = []
    list_search_comment = []
    list_search_subcomment = []

    # === Sequential note searches ===
    for kw in list_keywords:
        try:
            notes = delayed_spider_search_note(kw, cookies_str, xhs_spider, num_search)
            list_search_note.extend(notes)
        except Exception as e:
            logger.error(f"[ERROR] Note search for '{kw}' aborted: {e}")

    # === Sequential comment fetches ===
    for note in list_search_note:
        # cookie = next(cookie_cycle_comments)
        try:
            raw_comments = delayed_spider_search_comments(note, cookies_str, xhs_spider)
            comments, subcomments = handle_list_comment(raw_comments)
            list_search_comment.extend(comments)
            list_search_subcomment.extend(subcomments)
        except Exception as e:
            logger.error(f"[ERROR] Comments fetch aborted for note ID {note.get('id')}: {e}")

INFO:__main__:Searching keyword "ifastglobalbank", note count: 50


INFO:__main__:Retrieving note info - https://www.xiaohongshu.com/explore/67e29d0a000000000703645e?xsec_token=ABYNAWS8-LhXoQg-u2p4NdyrLbSRH0LwSGeDt0kL2m2kg=: True, msg: success


INFO:__main__:Retrieving note info - https://www.xiaohongshu.com/explore/6781e20e0000000017038ff5?xsec_token=ABeCQOIir9rjMRbIKYjfVc8hWFOTpg2Xjg1UUg_oE9gfY=: True, msg: success


INFO:__main__:Retrieving note info - https://www.xiaohongshu.com/explore/67b55e92000000002a00d220?xsec_token=ABm-vYGc-CIFwha5novRSrv4G024vESJ57KM6XyMb1ZHM=: True, msg: success


INFO:__main__:Retrieving note info - https://www.xiaohongshu.com/explore/680a2eae000000000b02fc17?xsec_token=ABPcwOmvei8nvj-s5tS-rFnt5CK4vIviBVYGwLFi9YbMk=: True, msg: success


...

ERROR:__main__:[delayed_spider_search_comments] all 3 attempts failed: RuntimeError('API returned success=False, msg=Exception("\'comments\'")')


ERROR:__main__:[ERROR] Comments fetch aborted for note ID None: API returned success=False, msg=Exception("'comments'")


INFO:__main__:Retrieving all comments - https://www.xiaohongshu.com/explore/680b49eb000000001e004e99?xsec_token=ABREVzzv7hWHwZd9s5DtNwnqunA1jGvbRuh5SjwMmizWs=: False, msg: 'comments'


INFO:__main__:Retrieving all comments - https://www.xiaohongshu.com/explore/680b49eb000000001e004e99?xsec_token=ABREVzzv7hWHwZd9s5DtNwnqunA1jGvbRuh5SjwMmizWs=: False, msg: 'comments'


KeyboardInterrupt: 

In [ ]:
folder_path = "datas/parquet_datas"
# Note info dataframe
schema_name = "mimir_igb_dpb_external_data.igb_xhs_scrape_data"
df_pandas_note_info = pd.DataFrame(list_search_note)
df_pandas_note_info = df_pandas_note_info.drop(columns=["avatar"])
df_pandas_note_info = df_pandas_note_info.rename(columns={"upload_time": "create_time", "nickname": "author_nickname", "user_url": "author_url"})
# df_pandas_note_info.columns = ['note_' + col if not col.startswith('note_') else col for col in df_pandas_note_info.columns]
df_pandas_note_info.to_parquet(f"{folder_path}/xhs_api_note_info.parquet", engine="pyarrow", index=False)



# Main Comment dataframe
schema_name = "mimir_igb_dpb_external_data.igb_xhs_scrape_data"
df_pandas_main_comment = pd.DataFrame(list_search_comment)
df_pandas_main_comment = df_pandas_main_comment.drop(columns=["pictures", "user_info", "at_users", "status", "sub_comment_cursor", "sub_comment_has_more", "show_tags", "sub_comments"])
df_pandas_main_comment["create_time"] = df_pandas_main_comment["create_time"].apply(lambda x: timestamp_to_str(x))
df_pandas_main_comment = df_pandas_main_comment.rename(columns={"comment_id": "main_comment_id", "content": "main_comment"})
# df_pandas_main_comment.columns = ['mc_' + col for col in df_pandas_main_comment.columns]

df_pandas_main_comment.to_parquet(f"{folder_path}/xhs_api_main_comment.parquet", engine="pyarrow", index=False)



# Sub Comment dataframe
schema_name = "mimir_igb_dpb_external_data.igb_xhs_scrape_data"
df_pandas_sub_comment = pd.DataFrame(list_search_subcomment)
df_pandas_sub_comment = df_pandas_sub_comment.drop(columns=["status", "user_info", "show_tags", "target_comment", "at_users", "pictures"])

df_pandas_sub_comment["create_time"] = df_pandas_sub_comment["create_time"].apply(lambda x: timestamp_to_str(x))
df_pandas_sub_comment = df_pandas_sub_comment.rename(columns={"comment_id": "main_comment_id", "content": "sub_comment"})
# df_pandas_sub_comment.columns = ['sc_' + col for col in df_pandas_sub_comment.columns]
df_pandas_sub_comment.to_parquet(f"{folder_path}/xhs_api_sub_comment.parquet", engine="pyarrow", index=False)


In [ ]:
df_pandas_sub_comment